In [1]:
# STEP 1: Install and import dependencies
!pip install -q transformers timm opencv-python
import torch, torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os, cv2, time
from IPython.display import Javascript, HTML, display
from base64 import b64decode, b64encode
from google.colab import output
from tqdm import tqdm




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.0 MB/s eta 0:00:00


In [2]:
import time
timestamp = time.strftime("%Y%m%d-%H%M%S")


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# STEP 2: Setup webcam capture JS
def save_video(base64_data):
    with open('video.mp4', 'wb') as f:
        f.write(b64decode(base64_data))
    print(" Video saved as video.mp4")
    input_drive_path = f'/content/drive/MyDrive/detr_input_video_{timestamp}.mp4'
    !cp /content/video.mp4 "$input_drive_path"
    print(f" Input video saved to: {input_drive_path}")


output.register_callback('notebook.saveVideo', save_video)

js = Javascript("""
async function recordVideo() {
  const div = document.createElement('div');
  const video = document.createElement('video');
  const startButton = document.createElement('button');
  const stopButton = document.createElement('button');

  startButton.textContent = 'Start Recording';
  stopButton.textContent = 'Stop Recording';
  stopButton.disabled = true;

  div.appendChild(video);
  div.appendChild(startButton);
  div.appendChild(stopButton);
  document.body.appendChild(div);

  const stream = await navigator.mediaDevices.getUserMedia({ video: true, audio: true });
  video.srcObject = stream;
  await video.play();

  let recorder = new MediaRecorder(stream);
  let data = [];

  recorder.ondataavailable = event => data.push(event.data);
  recorder.onstop = () => {
    const blob = new Blob(data, { type: 'video/webm' });
    const reader = new FileReader();
    reader.onloadend = () => {
      const base64data = reader.result.split(',')[1];
      google.colab.kernel.invokeFunction('notebook.saveVideo', [base64data], {});
    };
    reader.readAsDataURL(blob);
  };

  startButton.onclick = () => {
    recorder.start();
    startButton.disabled = true;
    stopButton.disabled = false;

  };

  stopButton.onclick = () => {
    recorder.stop();
    startButton.disabled = false;
    stopButton.disabled = true;
  };
}
recordVideo();
""")

display(js)

<IPython.core.display.Javascript object>

 Video saved as video.mp4
 Input video saved to: /content/drive/MyDrive/detr_input_video_20250723-100405.mp4


In [5]:
import cv2
import os

# Manually set the full video path
input_drive_path =  "/content/drive/MyDrive/detr_input_video_20250723-100405.mp4"

# Optional: confirm it exists
if not os.path.exists(input_drive_path):
    print("❌ File not found.")
else:
    cap = cv2.VideoCapture(input_drive_path)

    if not cap.isOpened():
        print("❌ Could not open video file.")
    else:
        # Get width and height
        width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        print(f"✅ Video resolution: {width}x{height}")

    cap.release()


✅ Video resolution: 640x480


In [6]:
# Step 3: Setup DETR model and functions
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
model.eval();



Downloading: "https://github.com/facebookresearch/detr/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 129MB/s]
Downloading: "https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth" to /root/.cache

In [7]:
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    return torch.stack([(x_c - 0.5 * w), (y_c - 0.5 * h),
                        (x_c + 0.5 * w), (y_c + 0.5 * h)], dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    return b * torch.tensor([img_w, img_h, img_w, img_h])

def filter_bboxes_from_outputs(outputs, size, threshold=0.9):
    probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > threshold
    probas_to_keep = probas[keep]
    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], size)
    return probas_to_keep, bboxes_scaled

CLASSES = ['N/A', 'person'] + ['N/A'] * 90



In [8]:
# Step 4: Extract frames from recorded video
video_path = "/content/video.mp4"
frames_dir = "/content/frames"
annotated_dir = "/content/annotated"
os.makedirs(frames_dir, exist_ok=True)
os.makedirs(annotated_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
interval = int(fps *0.25)
frame_count = 0
saved = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_count % interval == 0:
        h, w = frame.shape[:2]
        scale = 800 / min(h, w)
        new_size = (int(w * scale), int(h * scale))
        resized = cv2.resize(frame, new_size)
        out_path = os.path.join(frames_dir, f"frame_{saved:04d}.jpg")
        cv2.imwrite(out_path, resized)
        saved += 1
    frame_count += 1

cap.release()
print(f"Extracted and resized {saved} frames.")



Extracted and resized 72 frames.


In [9]:
# Step 5: Run DETR on frames
frame_paths = sorted(os.listdir(frames_dir))
for frame_name in tqdm(frame_paths):
    frame_path = os.path.join(frames_dir, frame_name)
    im = Image.open(frame_path).convert("RGB")
    img = transform(im).unsqueeze(0)
    # Should print [1, 3, H, W]

    with torch.no_grad():
        outputs = model(img)

    probas, bboxes = filter_bboxes_from_outputs(outputs, im.size)

    draw = ImageDraw.Draw(im)
    for p, (xmin, ymin, xmax, ymax) in zip(probas, bboxes.tolist()):
        cl = p.argmax().item()
        if CLASSES[cl] == 'person':
            draw.rectangle([xmin, ymin, xmax, ymax], outline='red', width=3)
            draw.text((xmin, ymin), f"person {p[cl]:.2f}", fill='red')

    im.save(os.path.join(annotated_dir, frame_name))

print(" Annotated frames saved.")





100%|██████████| 72/72 [04:48<00:00,  4.01s/it]

 Annotated frames saved.


In [13]:
# Step 6: Stitch annotated frames into output video
output_video_path = "/content/output_annotated_detr.mp4"
sample_frame = cv2.imread(os.path.join(annotated_dir, frame_paths[0]))
height, width = sample_frame.shape[:2]
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 4, (width, height))

for frame_name in frame_paths:
    frame = cv2.imread(os.path.join(annotated_dir, frame_name))
    out.write(frame)
out.release()

print(f" Output video saved to {output_video_path}")
output_drive_path = f'/content/drive/MyDrive/detr_output_annotated_{timestamp}.mp4'
!cp /content/output_annotated_detr.mp4 "$output_drive_path"
print(f" Output video saved to: {output_drive_path}")

 Output video saved to /content/output_annotated_detr.mp4
 Output video saved to: /content/drive/MyDrive/detr_output_annotated_20250723-100405.mp4


In [14]:
import glob
import os

# Find all DETR output videos in Drive
drive_videos = glob.glob('/content/drive/MyDrive/detr_output_annotated_*.mp4')

if not drive_videos:
    raise FileNotFoundError(" No DETR output videos found in Google Drive.")

# Get the most recently modified file
latest_drive_video = max(drive_videos, key=os.path.getmtime)
print(f" Latest video found: {latest_drive_video}")


 Latest video found: /content/drive/MyDrive/detr_output_annotated_20250723-100405.mp4


In [15]:
from IPython.display import HTML
from base64 import b64encode
import os

#  Input and output paths
#output_drive_path = f'/content/drive/MyDrive/detr_output_annotated_{timestamp}.mp4'
save_path = output_drive_path
          # Your original video (e.g., copied from Drive)
compressed_path = "/content/result_compressed.mp4"

#  Compress using ffmpeg (if not already compressed)
os.system(f"ffmpeg -y -i {save_path} -vcodec libx264 -crf 28 {compressed_path}")

#  Load and encode compressed video
with open(compressed_path, "rb") as f:
    video_data = f.read()
data_url = "data:video/mp4;base64," + b64encode(video_data).decode()

#  Display inline video
HTML(f"""
<video width="800" controls>
    <source src="{data_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
""")
